In [71]:
import math
import os
import re
from os import path

import spacy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_JSON = 'BauruJSON/'

nlp_exclude_list = ['parser',
                    'ner',
                    'entity_linker',
                    'entity_ruler',
                    'textcat',
                    'textcat_multilabel',
#                     'lemmatizer',
                    'trainable_lemmatizer',
                    'morphologizer',
                    'attribute_ruler',
                    'senter',
                    'sentencizer',
                    'tok2vec',
                    'transformer']
nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)

$TF_{ij} = \dfrac{f_{ij}}{\max_{k}f_{kj}}$ onde:
* $TF_{ij}$ é a "Frequência do termo _(Term Frequency)_" $i$ no documento $j$
* $f_{ij}$ é a frequência do termo $i$ no documento $j$

$IDF_{i} = \log_{2}\dfrac{N}{n_{i}}$ onde:
* $IDF_{i}$ é o "Inverso da Frequência em Documento _(Inverse Document Frequency)_" do termo $i$
* $N$ é o tamanho do Corpus
* $n_{i}$ é a quantidade de documentos de $N$ que o termo $i$ aparence

In [72]:
### Calcular o TF.IDF do Corpus de Publicações

def token_valido(token):
    """
    Verifica se um token é valido para ser incluído no saco de palavras
    
    :param token spacy.tokens.Token: token a ser verificado
    
    :returns: bool se é válido ou não
    """
    
    assert type(token) == spacy.tokens.Token
    
    return not (token.is_stop
               or token.is_space
               or token.is_punct
               or token.is_currency
               or token.is_quote
               or token.is_bracket
               or token.like_num
               or token.is_oov)

def calcula_tf(documento, norm = True):
    assert type(documento) == spacy.tokens.Doc
    
    saco = dict()
    
    for token in documento:
        if token_valido(token):
            chave = token.lemma_.lower()
            if chave not in saco:
                saco[chave] = 1
            else:
                saco[chave] += 1
    
    if len(saco) == 0:
        return saco
    
    if norm:
        chave_maxima = max(saco, key = saco.get)
        denominador = saco[chave_maxima]
        
        for chave in saco:
            saco[chave] = saco[chave] / float(denominador)

    return saco

def atualiza_ni(saco, n):
    for chave in saco:
        if chave not in n:
            n[chave] = 1
        else:
            n[chave] += 1
            
    return n

def calcula_tfidf(saco, n, N):
#     if type(termo) == str:
#         doc = nlp(termo)
#         if len(doc) > 0:
#             print('Documento com mais de um token. Escolhendo o primeiro')
#         chave = doc[0].norm_
#     elif type(termo) == spacy.tokens.Token:
#         chave = termo.norm_
    tfidf = []
    
    for chave in saco:
        tf = saco[chave]
        idf = math.log(N/n[chave])/math.log(2)
        tfidf.append((chave, tf * idf))
    
    return tfidf

In [97]:
N = 0
n = dict()
sacos = list()

for filename in tqdm(os.listdir(DIRETORIO_JSON)):
    with open(path.join(DIRETORIO_JSON, filename), 'r') as file:
        licitacao = json.load(file)
    if 'publicacoes' in licitacao:
        for publicacao in licitacao['publicacoes']:
#             if publicacao['titulo'] != 'Abertura':
#                 continue

            saco = calcula_tf(nlp(publicacao['conteudo']))
            sacos.append((licitacao['identificador'], saco, filename))
        
            atualiza_ni(saco, n)
            N += 1



100%|██████████| 3522/3522 [00:49<00:00, 70.74it/s]


In [75]:
documento = 0
df = pd.DataFrame(calcula_tfidf(sacos[documento][1], n, N), columns = ['termo', 'tfidf'])
df.nlargest(n = 10, columns = 'tfidf')
#df

,termo,tfidf
25,demanda,1.963397
26,judicial,1.963397
24,atendimento,1.479006
32,9h,1.370731
61,césar,1.360946
55,oc,1.350295
23,medicamentos,1.235027
62,leandro,1.149873
60,fernando,1.147769
35,pregoeira,1.109438


In [100]:
lista_documentos = []
#df = pd.DataFrame(columns = ['identificador', 'termo', 'tfidf'])
for d in range(len(sacos)):
    tfidf = calcula_tfidf(sacos[d][1], n, N)
    for termo in tfidf:
        lista_documentos.append({
            'identificador': sacos[d][0],
            'termo': termo[0],
            'tfidf': termo[1]
        })
df = pd.DataFrame(lista_documentos)
df[df['identificador'] == '3941'].nlargest(n = 50, columns = 'tfidf')

,identificador,termo,tfidf
4379,3941,07/02/2017,12.668885
4363,3941,pediátricas,3.694641
4369,3941,ginecológicos,3.694641
4370,3941,braçadeiras,3.694641
4410,3941,23/01/2017,3.694641
4332,3941,pag,3.448986
4360,3941,ginecológicas,3.448986
4372,3941,réguas,3.361307
4362,3941,eletrônicas,3.287177
4364,3941,focos,3.287177
